In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
from datetime import *
from sklearn import linear_model
from sklearn.preprocessing import *
from sklearn import ensemble
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import xgboost as xgb
from sklearn import metrics
from xgboost.sklearn import XGBClassifier  
from sklearn.model_selection import GridSearchCV,cross_val_score  
import matplotlib.pylab as plt  

In [19]:
#首先导入训练集数据进行线下数据集划分
train_df = pd.read_csv('chusai_train_df.csv')
# regex_string = ('user_gender_id_.*|user_age_level_.*|user_occupation_id_.*|user_star_level_.*' + 
#                                '|item_sales_level_.*|item_collected_level_.*|item_price_level_.*|item_pv_level_.*' + 
#                                '|shop_review_num_level_.*|shop_star_level_.*' + 
#                                '|context_page_id_.*' + 
#                                '|shop_review_positive_rate|shop_score_service|shop_score_delivery|shop_score_description' + 
#                '|hour')

regex_string = ('item_price_level_.*|item_pv_level_.*' + 
               '|shop_review_num_level_.*|shop_star_level_.*' + 
               '|context_page_id_.*')
fea = ['hour', 'user_gender_id', 'user_occupation_id', 'user_star_level', 'user_age_level', 'item_sales_level', 
       'item_brand_id', 'item_city_id', 'item_collected_level','item_price_level', 'item_pv_level', 
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
      'match_category_proportion', 'match_property_proportion', 
       'predict_category_number', 'predict_property_number', 
      'isFirstCategoryIn', 'isLastCategoryIn',
       'day_hour', 'weekday',
       'lastOneHour_sameItem_count', 'lastOneHour_sameFirstCategory_count', 'lastOneHour_sameLastCategory_count',
       'history_sameItem_count', 'history_sameFirstCategory_count', 'history_sameLastCategory_count',
       'is_history_sameItem', 'is_history_sameFirstCategory', 'is_history_sameLastCategory',
       'is_highProportion_brand', 'is_highSale_brand', 'category_number', 'property_number',
       'lastOneHour_sameBrand_count', 'lastOneHour_sameShop_count',
       'history_sameBrand_count', 'history_sameShop_count',
       'is_history_sameBrand', 'is_history_sameShop',
#        'is_history_firstClickItem', 
       'isLastOneHour_firstClickItem',
       
       'lastOneDay_sameItem_count', 'lastOneDay_sameFirstCategory_count', 'lastOneDay_sameLastCategory_count', 
       'lastOneDay_sameBrand_count', 'lastOneDay_sameShop_count', 
       
#        'item_sales_price', 'item_sales_collected', 'item_sales_pv',
#        'item_price_collected', 'item_price_pv', 'item_collected_pv',
#        'user_gender_age', 'user_gender_occupation', 'user_gender_star', 'user_age_occupation', 'user_age_star', 'user_occupation_star',
#        'shop_review_star',
       
       'shop_review_positive_rate_diff', 'shop_score_service_diff', 'shop_score_delivery_diff', 'shop_score_description_diff',
       'isHigh_shop_review_positive_rate', 'isHigh_shop_score_service', 'isHigh_shop_score_delivery', 'isHigh_shop_score_description',
       'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
#        'userItem_lastClickDeltaTime', 'userBrand_lastClickDeltaTime', 'userShop_lastClickDeltaTime', 'userFirstCategory_lastClickDeltaTime', 'userLastCategory_lastClickDeltaTime',
       'shop_item_classNumber', 'brand_item_classNumber', 'city_item_classNumber', 
       'shop_user_classNumber', 'brand_user_classNumber', 'city_user_classNumber',
#       'user_id_converse_smooth_rate', 'item_brand_id_converse_smooth_rate', 'item_id_converse_smooth_rate', 
#        'shop_id_converse_smooth_rate', 'item_city_id_converse_smooth_rate', 'real_first_category_converse_smooth_rate', 
#          'real_last_category_converse_smooth_rate', 
#        'user_item_converse_smooth_rate', 'user_brand_converse_smooth_rate', 'user_shop_converse_smooth_rate',
#          'user_city_converse_smooth_rate', 'user_first_category_converse_smooth_rate', 'user_last_category_converse_smooth_rate'
      ]

drop_fea = ['item_price_level_0', 'item_price_level_1', 'shop_star_level_lowHigh', #'is_history_sameItem', 
            'shop_star_level_middle', 'shop_review_num_level_lowHigh'
            ]
def dropFeture(drop_fea, df):
    for fea in drop_fea:
        df = df.drop(fea, 1)
    return df
train_df = dropFeture(drop_fea, train_df)

train_dataset = train_df.filter(regex = regex_string)
train_dataset = pd.concat([train_dataset, train_df[fea]], axis = 1)
train_dataset_label = train_df['is_trade']
print(len(train_dataset.columns))
print(train_dataset.columns)

test_df = pd.read_csv('chusai_test_df.csv')
test_dataset = test_df.filter(regex = regex_string)
test_dataset = pd.concat([test_dataset, test_df[fea]], axis = 1)
test_dataset = dropFeture(drop_fea, test_dataset)
print(len(test_dataset.columns))
print(test_dataset.columns)

#初步先用前6天数据作为线下训练集，用第七天数据作为线下测试集
train_dataset1 = train_df[(train_df.day <= 23) & (train_df.day >= 18)]
train_dataset1_x = train_dataset1.filter(regex = regex_string)
train_dataset1_x = pd.concat([train_dataset1_x, train_dataset1[fea]], axis = 1)
train_dataset1_y = train_dataset1['is_trade']
test_dataset1 = train_df[train_df.day == 24]
test_dataset1_x = test_dataset1.filter(regex = regex_string)
test_dataset1_x = pd.concat([test_dataset1_x, test_dataset1[fea]], axis = 1)
test_dataset1_y = test_dataset1['is_trade']

# train_dataset1 = train_df[train_df.day < 22]
# train_dataset1_x = train_dataset1.filter(regex = regex_string)
# train_dataset1_x = pd.concat([train_dataset1_x, train_dataset1[fea]], axis = 1)
# train_dataset1_y = train_dataset1['is_trade']
# test_dataset1 = train_df[train_df.day == 22]
# test_dataset1_x = test_dataset1.filter(regex = regex_string)
# test_dataset1_x = pd.concat([test_dataset1_x, test_dataset1[fea]], axis = 1)
# test_dataset1_y = test_dataset1['is_trade']
# test_dataset2 = train_df[train_df.day == 23]
# test_dataset2_x = test_dataset2.filter(regex = regex_string)
# test_dataset2_x = pd.concat([test_dataset2_x, test_dataset2[fea]], axis = 1)
# test_dataset2_y = test_dataset2['is_trade']


71
Index(['context_page_id_1', 'context_page_id_2', 'context_page_id_3',
       'context_page_id_4', 'item_pv_level_low', 'shop_star_level_loss',
       'hour', 'user_gender_id', 'user_occupation_id', 'user_star_level',
       'user_age_level', 'item_sales_level', 'item_brand_id', 'item_city_id',
       'item_collected_level', 'item_price_level', 'item_pv_level',
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
       'match_category_proportion', 'match_property_proportion',
       'predict_category_number', 'predict_property_number',
       'isFirstCategoryIn', 'isLastCategoryIn', 'day_hour', 'weekday',
       'lastOneHour_sameItem_count', 'lastOneHour_sameFirstCategory_count',
       'lastOneHour_sameLastCategory_count', 'history_sameItem_count',
       'history_sameFirstCategory_count', 'history_sameLastCategory_count',
       'is_history_sameItem', 'is_history_sameFirstCategory',
       'is_history_sameLastCategory', 'is_highProportion_brand',
       'is_highSa

In [27]:
#开始进行xgb模型训练
xgb_train_dataset1 = xgb.DMatrix(train_dataset1_x,label = train_dataset1_y)
xgb_test_dataset1 = xgb.DMatrix(test_dataset1_x, label = test_dataset1_y)

cv_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'gamma':0.7,
        'min_child_weight':11,
        'max_depth':4,
        'lambda':9,
        'subsample':0.9,
        'colsample_bytree':0.7,
        'colsample_bylevel':0.7,
        'eta': 0.01,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }

offline_watchlist = [(xgb_train_dataset1,'train'), (xgb_test_dataset1, 'val')]
offline_model = xgb.train(cv_params, xgb_train_dataset1, num_boost_round=3200, evals=offline_watchlist)

# params={'booster':'gbtree',
#         'objective': 'binary:logistic',
#         'eval_metric':'logloss',
#         'gamma':0.1,                               #6
#         'min_child_weight':5,                      #2
#         'max_depth':3,                             #1
#         'lambda':11,                               #5
#         'subsample':0.85,                          #3
#         'colsample_bytree':0.65,                    #4
#         'colsample_bylevel':0.7,
#         'scale_pos_weight':1,
#         'eta': 0.1,
#         'tree_method':'exact',
#         'seed':0,
#         'nthread':12
#         }

# offline_watchlist = [(xgb_train_dataset1,'train'), (xgb_test_dataset1, 'val')]
# offline_model = xgb.train(params, xgb_train_dataset1, num_boost_round=200, evals=offline_watchlist)

#predict test set
test_dataset1['label'] = offline_model.predict(xgb_test_dataset1)


[0]	train-logloss:0.683943	val-logloss:0.683895
[1]	train-logloss:0.674921	val-logloss:0.674827
[2]	train-logloss:0.666074	val-logloss:0.665933
[3]	train-logloss:0.657398	val-logloss:0.657213
[4]	train-logloss:0.648893	val-logloss:0.648668
[5]	train-logloss:0.640551	val-logloss:0.64028
[6]	train-logloss:0.632366	val-logloss:0.632053
[7]	train-logloss:0.624334	val-logloss:0.623978
[8]	train-logloss:0.616453	val-logloss:0.616062
[9]	train-logloss:0.608718	val-logloss:0.608283
[10]	train-logloss:0.601126	val-logloss:0.600648
[11]	train-logloss:0.593671	val-logloss:0.593151
[12]	train-logloss:0.586349	val-logloss:0.585787
[13]	train-logloss:0.579157	val-logloss:0.578559
[14]	train-logloss:0.572099	val-logloss:0.57146
[15]	train-logloss:0.565162	val-logloss:0.564483
[16]	train-logloss:0.558349	val-logloss:0.55763
[17]	train-logloss:0.551657	val-logloss:0.550899
[18]	train-logloss:0.545079	val-logloss:0.544286
[19]	train-logloss:0.538614	val-logloss:0.537782
[20]	train-logloss:0.532259	val-l

[167]	train-logloss:0.154253	val-logloss:0.149584
[168]	train-logloss:0.153422	val-logloss:0.148733
[169]	train-logloss:0.152603	val-logloss:0.147894
[170]	train-logloss:0.151793	val-logloss:0.147064
[171]	train-logloss:0.150993	val-logloss:0.146244
[172]	train-logloss:0.150204	val-logloss:0.145436
[173]	train-logloss:0.149424	val-logloss:0.144635
[174]	train-logloss:0.148655	val-logloss:0.143846
[175]	train-logloss:0.147897	val-logloss:0.143068
[176]	train-logloss:0.147147	val-logloss:0.142299
[177]	train-logloss:0.146406	val-logloss:0.141541
[178]	train-logloss:0.145677	val-logloss:0.140792
[179]	train-logloss:0.144954	val-logloss:0.140051
[180]	train-logloss:0.14424	val-logloss:0.139317
[181]	train-logloss:0.143535	val-logloss:0.138594
[182]	train-logloss:0.142839	val-logloss:0.13788
[183]	train-logloss:0.142151	val-logloss:0.137173
[184]	train-logloss:0.141473	val-logloss:0.136477
[185]	train-logloss:0.140804	val-logloss:0.13579
[186]	train-logloss:0.140141	val-logloss:0.135109
[18

[332]	train-logloss:0.096934	val-logloss:0.089973
[333]	train-logloss:0.096839	val-logloss:0.08987
[334]	train-logloss:0.096744	val-logloss:0.089766
[335]	train-logloss:0.096652	val-logloss:0.089665
[336]	train-logloss:0.09656	val-logloss:0.089567
[337]	train-logloss:0.09647	val-logloss:0.089469
[338]	train-logloss:0.096381	val-logloss:0.089372
[339]	train-logloss:0.096293	val-logloss:0.089277
[340]	train-logloss:0.096207	val-logloss:0.089186
[341]	train-logloss:0.096121	val-logloss:0.089093
[342]	train-logloss:0.096038	val-logloss:0.089003
[343]	train-logloss:0.095956	val-logloss:0.088914
[344]	train-logloss:0.095873	val-logloss:0.088823
[345]	train-logloss:0.095792	val-logloss:0.088734
[346]	train-logloss:0.095714	val-logloss:0.088649
[347]	train-logloss:0.095635	val-logloss:0.088563
[348]	train-logloss:0.095556	val-logloss:0.088476
[349]	train-logloss:0.095481	val-logloss:0.088395
[350]	train-logloss:0.095405	val-logloss:0.088312
[351]	train-logloss:0.09533	val-logloss:0.08823
[352]

[497]	train-logloss:0.090534	val-logloss:0.083006
[498]	train-logloss:0.090521	val-logloss:0.082994
[499]	train-logloss:0.090509	val-logloss:0.082984
[500]	train-logloss:0.090498	val-logloss:0.08297
[501]	train-logloss:0.090486	val-logloss:0.082957
[502]	train-logloss:0.090473	val-logloss:0.082944
[503]	train-logloss:0.090462	val-logloss:0.082933
[504]	train-logloss:0.09045	val-logloss:0.082921
[505]	train-logloss:0.090438	val-logloss:0.082909
[506]	train-logloss:0.090426	val-logloss:0.082897
[507]	train-logloss:0.090414	val-logloss:0.082885
[508]	train-logloss:0.090402	val-logloss:0.082873
[509]	train-logloss:0.090391	val-logloss:0.082864
[510]	train-logloss:0.090381	val-logloss:0.082854
[511]	train-logloss:0.090371	val-logloss:0.082842
[512]	train-logloss:0.090361	val-logloss:0.082832
[513]	train-logloss:0.09035	val-logloss:0.082821
[514]	train-logloss:0.090338	val-logloss:0.082811
[515]	train-logloss:0.090327	val-logloss:0.082802
[516]	train-logloss:0.090317	val-logloss:0.082792
[51

[662]	train-logloss:0.089343	val-logloss:0.081983
[663]	train-logloss:0.089339	val-logloss:0.081982
[664]	train-logloss:0.089335	val-logloss:0.08198
[665]	train-logloss:0.089331	val-logloss:0.081978
[666]	train-logloss:0.089326	val-logloss:0.081976
[667]	train-logloss:0.089321	val-logloss:0.081973
[668]	train-logloss:0.089317	val-logloss:0.081971
[669]	train-logloss:0.089313	val-logloss:0.081967
[670]	train-logloss:0.089309	val-logloss:0.081965
[671]	train-logloss:0.089304	val-logloss:0.081961
[672]	train-logloss:0.0893	val-logloss:0.081958
[673]	train-logloss:0.089296	val-logloss:0.081956
[674]	train-logloss:0.089291	val-logloss:0.081954
[675]	train-logloss:0.089287	val-logloss:0.081952
[676]	train-logloss:0.089283	val-logloss:0.08195
[677]	train-logloss:0.08928	val-logloss:0.081945
[678]	train-logloss:0.089276	val-logloss:0.081942
[679]	train-logloss:0.089272	val-logloss:0.081939
[680]	train-logloss:0.089266	val-logloss:0.081934
[681]	train-logloss:0.089261	val-logloss:0.081929
[682]

...

[2611]	train-logloss:0.085921	val-logloss:0.081101
[2612]	train-logloss:0.085919	val-logloss:0.081102
[2613]	train-logloss:0.085918	val-logloss:0.081102
[2614]	train-logloss:0.085917	val-logloss:0.081102
[2615]	train-logloss:0.085917	val-logloss:0.081102
[2616]	train-logloss:0.085916	val-logloss:0.081102
[2617]	train-logloss:0.085915	val-logloss:0.081103
[2618]	train-logloss:0.085913	val-logloss:0.081103
[2619]	train-logloss:0.085913	val-logloss:0.081102
[2620]	train-logloss:0.085912	val-logloss:0.081103
[2621]	train-logloss:0.08591	val-logloss:0.081102
[2622]	train-logloss:0.085909	val-logloss:0.081102
[2623]	train-logloss:0.085909	val-logloss:0.081101
[2624]	train-logloss:0.085907	val-logloss:0.081102
[2625]	train-logloss:0.085907	val-logloss:0.081102
[2626]	train-logloss:0.085906	val-logloss:0.081102
[2627]	train-logloss:0.085905	val-logloss:0.081102
[2628]	train-logloss:0.085904	val-logloss:0.081101
[2629]	train-logloss:0.085902	val-logloss:0.081101
[2630]	train-logloss:0.085901	va

[2773]	train-logloss:0.085741	val-logloss:0.08109
[2774]	train-logloss:0.08574	val-logloss:0.08109
[2775]	train-logloss:0.085738	val-logloss:0.081091
[2776]	train-logloss:0.085737	val-logloss:0.08109
[2777]	train-logloss:0.085736	val-logloss:0.08109
[2778]	train-logloss:0.085734	val-logloss:0.08109
[2779]	train-logloss:0.085733	val-logloss:0.081089
[2780]	train-logloss:0.085732	val-logloss:0.081089
[2781]	train-logloss:0.085731	val-logloss:0.08109
[2782]	train-logloss:0.08573	val-logloss:0.08109
[2783]	train-logloss:0.085729	val-logloss:0.081091
[2784]	train-logloss:0.085728	val-logloss:0.081091
[2785]	train-logloss:0.085727	val-logloss:0.081091
[2786]	train-logloss:0.085726	val-logloss:0.081091
[2787]	train-logloss:0.085725	val-logloss:0.081091
[2788]	train-logloss:0.085723	val-logloss:0.081093
[2789]	train-logloss:0.085721	val-logloss:0.081093
[2790]	train-logloss:0.085721	val-logloss:0.081095
[2791]	train-logloss:0.08572	val-logloss:0.081094
[2792]	train-logloss:0.085718	val-logloss

[2935]	train-logloss:0.085566	val-logloss:0.081079
[2936]	train-logloss:0.085565	val-logloss:0.081079
[2937]	train-logloss:0.085564	val-logloss:0.081079
[2938]	train-logloss:0.085563	val-logloss:0.081079
[2939]	train-logloss:0.085563	val-logloss:0.081079
[2940]	train-logloss:0.085561	val-logloss:0.081079
[2941]	train-logloss:0.085561	val-logloss:0.081079
[2942]	train-logloss:0.08556	val-logloss:0.081078
[2943]	train-logloss:0.085559	val-logloss:0.081078
[2944]	train-logloss:0.085558	val-logloss:0.081078
[2945]	train-logloss:0.085556	val-logloss:0.081079
[2946]	train-logloss:0.085555	val-logloss:0.081079
[2947]	train-logloss:0.085554	val-logloss:0.081079
[2948]	train-logloss:0.085553	val-logloss:0.081079
[2949]	train-logloss:0.085553	val-logloss:0.081079
[2950]	train-logloss:0.085551	val-logloss:0.08108
[2951]	train-logloss:0.08555	val-logloss:0.08108
[2952]	train-logloss:0.085549	val-logloss:0.08108
[2953]	train-logloss:0.085548	val-logloss:0.081079
[2954]	train-logloss:0.085547	val-lo

[3097]	train-logloss:0.085397	val-logloss:0.081069
[3098]	train-logloss:0.085397	val-logloss:0.08107
[3099]	train-logloss:0.085395	val-logloss:0.081069
[3100]	train-logloss:0.085394	val-logloss:0.081069
[3101]	train-logloss:0.085392	val-logloss:0.081069
[3102]	train-logloss:0.085391	val-logloss:0.081068
[3103]	train-logloss:0.08539	val-logloss:0.081069
[3104]	train-logloss:0.085389	val-logloss:0.081069
[3105]	train-logloss:0.085389	val-logloss:0.08107
[3106]	train-logloss:0.085388	val-logloss:0.08107
[3107]	train-logloss:0.085386	val-logloss:0.08107
[3108]	train-logloss:0.085385	val-logloss:0.08107
[3109]	train-logloss:0.085385	val-logloss:0.081071
[3110]	train-logloss:0.085384	val-logloss:0.08107
[3111]	train-logloss:0.085383	val-logloss:0.08107
[3112]	train-logloss:0.085383	val-logloss:0.08107
[3113]	train-logloss:0.085382	val-logloss:0.08107
[3114]	train-logloss:0.085381	val-logloss:0.08107
[3115]	train-logloss:0.08538	val-logloss:0.08107
[3116]	train-logloss:0.085379	val-logloss:0.

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
#输出线下训练的xgb模型的重要性
feat_imp = pd.Series(offline_model.get_fscore()).sort_values(ascending=False)
print(feat_imp.head(100))

shop_score_delivery                    2098
item_sales_level                       2069
shop_score_description                 2041
shop_score_delivery_diff               1760
property_number                        1662
hour                                   1650
shop_score_service                     1626
shop_score_description_diff            1588
shop_review_positive_rate_diff         1578
shop_score_service_diff                1551
item_brand_id                          1369
item_price_level                       1179
shop_review_positive_rate              1173
user_star_level                        1130
item_city_id                           1092
user_age_level                          950
item_collected_level                    941
item_pv_level                           843
match_property_proportion               817
shop_review_num_level                   761
match_category_proportion               736
predict_category_number                 713
predict_property_number         

In [29]:
#计算线下误差
def calError(col1, col2):
    error = metrics.log_loss(col1, col2)
    return error

dataset1_error = calError(test_dataset1_y.values, test_dataset1['label'].values)
print(test_dataset1_y.values[:10])
print(test_dataset1[['label', 'is_trade']][test_dataset1.is_trade == 1].values[:10])
print('dataset1_error : ', dataset1_error)

[0 0 0 0 0 0 0 1 0 0]
[[ 0.02588711  1.        ]
 [ 0.00990827  1.        ]
 [ 0.03177946  1.        ]
 [ 0.02094332  1.        ]
 [ 0.04839975  1.        ]
 [ 0.01696548  1.        ]
 [ 0.08621936  1.        ]
 [ 0.07269783  1.        ]
 [ 0.00546336  1.        ]
 [ 0.0291548   1.        ]]
dataset1_error :  0.0810680123898


In [30]:
#进行正式模型训练
xgb_train = xgb.DMatrix(train_dataset,label=train_dataset_label)
xgb_test = xgb.DMatrix(test_dataset)

cv_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'gamma':0.7,
        'min_child_weight':11,
        'max_depth':4,
        'lambda':9,
        'subsample':0.9,
        'colsample_bytree':0.7,
        'colsample_bylevel':0.7,
        'eta': 0.01,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }

watchlist = [(xgb_train,'train')]
model = xgb.train(cv_params, xgb_train, num_boost_round=3200, evals=watchlist)

# params={'booster':'gbtree',
#         'objective': 'binary:logistic',
#         'eval_metric':'logloss',
#         'gamma':0.1,                               #6
#         'min_child_weight':5,                      #2
#         'max_depth':3,                             #1
#         'lambda':11,                               #5
#         'subsample':0.85,                          #3
#         'colsample_bytree':0.65,                    #4
#         'colsample_bylevel':0.7,
#         'scale_pos_weight':1,
#         'eta': 0.1,
#         'tree_method':'exact',
#         'seed':0,
#         'nthread':12
#         }

# watchlist = [(xgb_train,'train')]
# model = xgb.train(params, xgb_train, num_boost_round=200, evals=watchlist)



[0]	train-logloss:0.683934
[1]	train-logloss:0.6749
[2]	train-logloss:0.666049
[3]	train-logloss:0.657366
[4]	train-logloss:0.648849
[5]	train-logloss:0.640497
[6]	train-logloss:0.6323
[7]	train-logloss:0.62426
[8]	train-logloss:0.616369
[9]	train-logloss:0.608627
[10]	train-logloss:0.601025
[11]	train-logloss:0.593561
[12]	train-logloss:0.586232
[13]	train-logloss:0.579035
[14]	train-logloss:0.571967
[15]	train-logloss:0.565023
[16]	train-logloss:0.558201
[17]	train-logloss:0.551498
[18]	train-logloss:0.54491
[19]	train-logloss:0.538436
[20]	train-logloss:0.532073
[21]	train-logloss:0.525819
[22]	train-logloss:0.519669
[23]	train-logloss:0.513625
[24]	train-logloss:0.50768
[25]	train-logloss:0.501835
[26]	train-logloss:0.496087
[27]	train-logloss:0.490431
[28]	train-logloss:0.484867
[29]	train-logloss:0.479394
[30]	train-logloss:0.474011
[31]	train-logloss:0.468713
[32]	train-logloss:0.463499
[33]	train-logloss:0.458368
[34]	train-logloss:0.453321
[35]	train-logloss:0.448353
[36]	trai

[288]	train-logloss:0.101787
[289]	train-logloss:0.101612
[290]	train-logloss:0.10144
[291]	train-logloss:0.10127
[292]	train-logloss:0.101102
[293]	train-logloss:0.100935
[294]	train-logloss:0.100774
[295]	train-logloss:0.100616
[296]	train-logloss:0.100461
[297]	train-logloss:0.100302
[298]	train-logloss:0.100148
[299]	train-logloss:0.099995
[300]	train-logloss:0.099848
[301]	train-logloss:0.099699
[302]	train-logloss:0.099553
[303]	train-logloss:0.099409
[304]	train-logloss:0.099267
[305]	train-logloss:0.099127
[306]	train-logloss:0.098988
[307]	train-logloss:0.098852
[308]	train-logloss:0.098718
[309]	train-logloss:0.098584
[310]	train-logloss:0.098452
[311]	train-logloss:0.098322
[312]	train-logloss:0.098193
[313]	train-logloss:0.098066
[314]	train-logloss:0.097941
[315]	train-logloss:0.097817
[316]	train-logloss:0.097696
[317]	train-logloss:0.097576
[318]	train-logloss:0.097457
[319]	train-logloss:0.097344
[320]	train-logloss:0.097229
[321]	train-logloss:0.097117
[322]	train-logl

[572]	train-logloss:0.088929
[573]	train-logloss:0.088922
[574]	train-logloss:0.088915
[575]	train-logloss:0.088908
[576]	train-logloss:0.088902
[577]	train-logloss:0.088896
[578]	train-logloss:0.088889
[579]	train-logloss:0.088883
[580]	train-logloss:0.088877
[581]	train-logloss:0.088871
[582]	train-logloss:0.088864
[583]	train-logloss:0.088857
[584]	train-logloss:0.088851
[585]	train-logloss:0.088845
[586]	train-logloss:0.08884
[587]	train-logloss:0.088834
[588]	train-logloss:0.088828
[589]	train-logloss:0.088822
[590]	train-logloss:0.088815
[591]	train-logloss:0.088809
[592]	train-logloss:0.088803
[593]	train-logloss:0.088797
[594]	train-logloss:0.088792
[595]	train-logloss:0.088786
[596]	train-logloss:0.088781
[597]	train-logloss:0.088775
[598]	train-logloss:0.088769
[599]	train-logloss:0.088763
[600]	train-logloss:0.088757
[601]	train-logloss:0.088751
[602]	train-logloss:0.088746
[603]	train-logloss:0.08874
[604]	train-logloss:0.088733
[605]	train-logloss:0.088726
[606]	train-logl

[856]	train-logloss:0.087778
[857]	train-logloss:0.087774
[858]	train-logloss:0.087771
[859]	train-logloss:0.087769
[860]	train-logloss:0.087766
[861]	train-logloss:0.087764
[862]	train-logloss:0.087762
[863]	train-logloss:0.08776
[864]	train-logloss:0.087757
[865]	train-logloss:0.087755
[866]	train-logloss:0.087753
[867]	train-logloss:0.08775
[868]	train-logloss:0.087747
[869]	train-logloss:0.087744
[870]	train-logloss:0.087741
[871]	train-logloss:0.087738
[872]	train-logloss:0.087735
[873]	train-logloss:0.087733
[874]	train-logloss:0.08773
[875]	train-logloss:0.087728
[876]	train-logloss:0.087725
[877]	train-logloss:0.087722
[878]	train-logloss:0.087719
[879]	train-logloss:0.087717
[880]	train-logloss:0.087714
[881]	train-logloss:0.087711
[882]	train-logloss:0.087709
[883]	train-logloss:0.087706
[884]	train-logloss:0.087703
[885]	train-logloss:0.087701
[886]	train-logloss:0.087698
[887]	train-logloss:0.087695
[888]	train-logloss:0.087691
[889]	train-logloss:0.087689
[890]	train-loglo

[1135]	train-logloss:0.087135
[1136]	train-logloss:0.087134
[1137]	train-logloss:0.087131
[1138]	train-logloss:0.08713
[1139]	train-logloss:0.087127
[1140]	train-logloss:0.087125
[1141]	train-logloss:0.087123
[1142]	train-logloss:0.087122
[1143]	train-logloss:0.087121
[1144]	train-logloss:0.087119
[1145]	train-logloss:0.087117
[1146]	train-logloss:0.087116
[1147]	train-logloss:0.087115
[1148]	train-logloss:0.087114
[1149]	train-logloss:0.087112
[1150]	train-logloss:0.087111
[1151]	train-logloss:0.087109
[1152]	train-logloss:0.087107
[1153]	train-logloss:0.087104
[1154]	train-logloss:0.087102
[1155]	train-logloss:0.0871
[1156]	train-logloss:0.087097
[1157]	train-logloss:0.087095
[1158]	train-logloss:0.087094
[1159]	train-logloss:0.087092
[1160]	train-logloss:0.087091
[1161]	train-logloss:0.087089
[1162]	train-logloss:0.087087
[1163]	train-logloss:0.087085
[1164]	train-logloss:0.087082
[1165]	train-logloss:0.087081
[1166]	train-logloss:0.087079
[1167]	train-logloss:0.087078
[1168]	train-

...

[1959]	train-logloss:0.085942
[1960]	train-logloss:0.085941
[1961]	train-logloss:0.08594
[1962]	train-logloss:0.08594
[1963]	train-logloss:0.085939
[1964]	train-logloss:0.085938
[1965]	train-logloss:0.085937
[1966]	train-logloss:0.085936
[1967]	train-logloss:0.085934
[1968]	train-logloss:0.085933
[1969]	train-logloss:0.085933
[1970]	train-logloss:0.085932
[1971]	train-logloss:0.085931
[1972]	train-logloss:0.08593
[1973]	train-logloss:0.085928
[1974]	train-logloss:0.085927
[1975]	train-logloss:0.085927
[1976]	train-logloss:0.085926
[1977]	train-logloss:0.085925
[1978]	train-logloss:0.085925
[1979]	train-logloss:0.085923
[1980]	train-logloss:0.085921
[1981]	train-logloss:0.085921
[1982]	train-logloss:0.085919
[1983]	train-logloss:0.085918
[1984]	train-logloss:0.085917
[1985]	train-logloss:0.085915
[1986]	train-logloss:0.085914
[1987]	train-logloss:0.085913
[1988]	train-logloss:0.085912
[1989]	train-logloss:0.085911
[1990]	train-logloss:0.085909
[1991]	train-logloss:0.085908
[1992]	train-

[2233]	train-logloss:0.08563
[2234]	train-logloss:0.085629
[2235]	train-logloss:0.085628
[2236]	train-logloss:0.085627
[2237]	train-logloss:0.085626
[2238]	train-logloss:0.085625
[2239]	train-logloss:0.085623
[2240]	train-logloss:0.085622
[2241]	train-logloss:0.085622
[2242]	train-logloss:0.085621
[2243]	train-logloss:0.08562
[2244]	train-logloss:0.08562
[2245]	train-logloss:0.085619
[2246]	train-logloss:0.085618
[2247]	train-logloss:0.085616
[2248]	train-logloss:0.085616
[2249]	train-logloss:0.085615
[2250]	train-logloss:0.085614
[2251]	train-logloss:0.085613
[2252]	train-logloss:0.085612
[2253]	train-logloss:0.08561
[2254]	train-logloss:0.085609
[2255]	train-logloss:0.085608
[2256]	train-logloss:0.085606
[2257]	train-logloss:0.085605
[2258]	train-logloss:0.085604
[2259]	train-logloss:0.085604
[2260]	train-logloss:0.085602
[2261]	train-logloss:0.085602
[2262]	train-logloss:0.085602
[2263]	train-logloss:0.0856
[2264]	train-logloss:0.085599
[2265]	train-logloss:0.085598
[2266]	train-log

[2508]	train-logloss:0.085331
[2509]	train-logloss:0.08533
[2510]	train-logloss:0.085329
[2511]	train-logloss:0.085328
[2512]	train-logloss:0.085326
[2513]	train-logloss:0.085325
[2514]	train-logloss:0.085324
[2515]	train-logloss:0.085323
[2516]	train-logloss:0.085322
[2517]	train-logloss:0.085322
[2518]	train-logloss:0.085321
[2519]	train-logloss:0.08532
[2520]	train-logloss:0.085318
[2521]	train-logloss:0.085318
[2522]	train-logloss:0.085317
[2523]	train-logloss:0.085316
[2524]	train-logloss:0.085315
[2525]	train-logloss:0.085315
[2526]	train-logloss:0.085314
[2527]	train-logloss:0.085312
[2528]	train-logloss:0.085311
[2529]	train-logloss:0.08531
[2530]	train-logloss:0.085308
[2531]	train-logloss:0.085307
[2532]	train-logloss:0.085306
[2533]	train-logloss:0.085306
[2534]	train-logloss:0.085304
[2535]	train-logloss:0.085304
[2536]	train-logloss:0.085303
[2537]	train-logloss:0.085301
[2538]	train-logloss:0.0853
[2539]	train-logloss:0.085299
[2540]	train-logloss:0.085298
[2541]	train-lo

[2783]	train-logloss:0.085058
[2784]	train-logloss:0.085057
[2785]	train-logloss:0.085055
[2786]	train-logloss:0.085054
[2787]	train-logloss:0.085053
[2788]	train-logloss:0.085052
[2789]	train-logloss:0.08505
[2790]	train-logloss:0.085049
[2791]	train-logloss:0.085047
[2792]	train-logloss:0.085046
[2793]	train-logloss:0.085045
[2794]	train-logloss:0.085044
[2795]	train-logloss:0.085042
[2796]	train-logloss:0.085042
[2797]	train-logloss:0.085041
[2798]	train-logloss:0.085041
[2799]	train-logloss:0.08504
[2800]	train-logloss:0.085039
[2801]	train-logloss:0.085038
[2802]	train-logloss:0.085037
[2803]	train-logloss:0.085036
[2804]	train-logloss:0.085034
[2805]	train-logloss:0.085033
[2806]	train-logloss:0.085032
[2807]	train-logloss:0.085031
[2808]	train-logloss:0.08503
[2809]	train-logloss:0.085029
[2810]	train-logloss:0.085028
[2811]	train-logloss:0.085027
[2812]	train-logloss:0.085026
[2813]	train-logloss:0.085025
[2814]	train-logloss:0.085023
[2815]	train-logloss:0.085022
[2816]	train-

[3058]	train-logloss:0.084798
[3059]	train-logloss:0.084797
[3060]	train-logloss:0.084796
[3061]	train-logloss:0.084796
[3062]	train-logloss:0.084794
[3063]	train-logloss:0.084793
[3064]	train-logloss:0.084792
[3065]	train-logloss:0.084791
[3066]	train-logloss:0.084789
[3067]	train-logloss:0.084789
[3068]	train-logloss:0.084788
[3069]	train-logloss:0.084787
[3070]	train-logloss:0.084786
[3071]	train-logloss:0.084786
[3072]	train-logloss:0.084785
[3073]	train-logloss:0.084784
[3074]	train-logloss:0.084782
[3075]	train-logloss:0.084781
[3076]	train-logloss:0.08478
[3077]	train-logloss:0.08478
[3078]	train-logloss:0.084779
[3079]	train-logloss:0.084779
[3080]	train-logloss:0.084777
[3081]	train-logloss:0.084776
[3082]	train-logloss:0.084776
[3083]	train-logloss:0.084775
[3084]	train-logloss:0.084774
[3085]	train-logloss:0.084773
[3086]	train-logloss:0.084773
[3087]	train-logloss:0.084772
[3088]	train-logloss:0.08477
[3089]	train-logloss:0.08477
[3090]	train-logloss:0.084769
[3091]	train-l

In [31]:
#predict test set
test_df['predicted_score'] = model.predict(xgb_test)

In [34]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../../data/chusai/submit/%s.txt' % fileName, sep=' ', header=True, index=False)

print(test_df[['instance_id', 'predicted_score']].head(10))
exportResult(test_df[['instance_id', 'predicted_score']], 'chusai_xgb_4_19')

        instance_id  predicted_score
0    55144604125888         0.023613
1   221669167755726         0.006441
2   566644865989395         0.029448
3   954943998950521         0.011380
4  1192015136416062         0.017742
5  1239632009133923         0.009072
6  1370758425798224         0.014657
7  1386245957915884         0.010040
8  1587829942816220         0.012812
9  1965296257027358         0.010851


In [33]:
print(len(test_df[['instance_id', 'predicted_score']][test_df.predicted_score > 0.1]))
print(test_df['predicted_score'].mean())

121
0.01805718243122101
